## Imports
There are just a few imports for this analysis, but this version will deviate from the Soccermatics approach due to the mplsoccer library throwing and error when making a call to Statsbomb.  Instead, the statsbomb data will be called directly for the dataframe.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen, VerticalPitch
from statsbombpy import sb

## Opening the dataset
Using the sb parser in place of SBopen in mplsoccer the event data is pulled out. Calling on the event level data and inputting the id of the game as a parameter loads the game data. The event data is stored in a dataframe df. From this dataframe take out the names of the two teams. Then, filter the dataframe so that only shots are left in the events.

In [6]:
# get event data
event_lib = sb.events(match_id=69301, split=True, flatten_attrs=False)
event_lib.keys()

dict_keys(['starting_xis', 'half_starts', 'passes', 'ball_receipts', 'carrys', 'pressures', 'ball_recoverys', 'clearances', 'blocks', 'dispossesseds', 'duels', 'dribbled_pasts', 'dribbles', 'interceptions', 'goal_keepers', 'miscontrols', 'foul_committeds', 'foul_wons', 'injury_stoppages', 'errors', 'shots', 'substitutions', 'half_ends', 'tactical_shifts', 'bad_behaviours'])

In [14]:
shots = event_lib['shots']
shots.head(2)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,position,location,duration,related_events,shot,match_id,possession_team_id,team_id,player_id,out
0,f158cbc0-a7a1-4787-8f73-15c66ae55764,340,1,00:10:24.396,10,24,Shot,18,Sweden Women's,From Free Kick,...,Center Attacking Midfield,"[104.7, 45.5]",1.342346,[034cfcf5-cc65-4850-9134-32c87320bad9],"{'statsbomb_xg': 0.1051876, 'end_location': [1...",69301,858,858,10220,NaN
1,de5c41c5-9200-439e-b051-bfe2016d689a,531,1,00:15:28.174,15,28,Shot,25,Sweden Women's,From Free Kick,...,Right Wing,"[110.9, 53.1]",0.472900,[beb4be84-907c-4f48-93b2-69310e6af8fd],"{'statsbomb_xg': 0.039003637, 'end_location': ...",69301,858,858,13881,NaN


In [15]:
#get team names
team1, team2 = shots.possession_team.unique()
print(team1, team2)

Sweden Women's England Women's


In [16]:
shots_id = (shots.set_index('id'))